In [2]:
%load_ext autoreload
%autoreload 2
%cd ~/Research/Sriram/DeepSetRNN

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/minh/Research/Sriram/DeepSetRNN


In [3]:
from collections import defaultdict, namedtuple
import os
import string
import logging
import importlib

import pandas as pd
from sklearn import metrics
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from src import experiment_utils, utils

In [4]:
logging.info("starting logger")
_LOGGER = logging.getLogger('VisitTimeSeries')
_LOGGER.setLevel(logging.DEBUG)

# Reading data

In [5]:
_ROOT_DATA_PATH = 'data/MIMIC3database/'
_PROCESSED_DATA_PATH = os.path.join(_ROOT_DATA_PATH, 'processed/MIMIC3EachPerson')

In [140]:
train_series_df = pd.read_csv(os.path.join(_PROCESSED_DATA_PATH, 'train_timeseries.csv'),
                              index_col=0)

train_labels_df = pd.read_csv(os.path.join(_PROCESSED_DATA_PATH, 'train_labels.csv'),
                              index_col=0)

train_labels_df.drop(['Icustay', 'Ethnicity', 'Gender', 'Age', 'Height', 'Weight',
       'Length of Stay', 'Mortality'], axis=1, inplace=True)


train_series_df = train_series_df.set_index(['SUBJECT_ID', 'ADMISSION_NUM'])
train_labels_df = train_labels_df.set_index(['SUBJECT_ID', 'ADMISSION_NUM'])
train_indices = train_series_df.index

/home/minh/.conda/envs/deepset/lib/python3.6/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [141]:
test_series_df = pd.read_csv(os.path.join(_PROCESSED_DATA_PATH, 'test_timeseries.csv'),
                              index_col=0)

test_labels_df = pd.read_csv(os.path.join(_PROCESSED_DATA_PATH, 'test_labels.csv'),
                              index_col=0)

test_labels_df.drop(['Icustay', 'Ethnicity', 'Gender', 'Age', 'Height', 'Weight',
       'Length of Stay', 'Mortality'], axis=1, inplace=True)

test_series_df = test_series_df.set_index(['SUBJECT_ID', 'ADMISSION_NUM'])
test_labels_df = test_labels_df.set_index(['SUBJECT_ID', 'ADMISSION_NUM'])
test_indices = test_series_df.index

In [143]:
set(train_series_df.index).difference(set(train_labels_df.index))

{(12607, 2), (17269, 2), (17796, 2), (23407, 1), (24975, 2)}

In [128]:
n_features = train_series_df.iloc[0].shape[0]
n_class = train_labels_df.iloc[0].shape[0]

# Training Model

In [131]:
_MODEL_LOG_ROOT_PATH = 'logs/MIMIC3/VisitTimeSeries'
_MODEL_ROOT_PATH = 'models/MIMIC3/VisitTimeSeries'

In [132]:
from src.model.lstm import LSTMClassifier

ModelArgs = namedtuple('HospitalVisitLSTMClassifier', 
                      ['hidden_dims',
                       'n_epoch',
                       'lr',
                       'momentum',
                      'n_layers',
                      'dropout']
                      )
args = ModelArgs(
    hidden_dims=[1000,500],
    n_epoch = 10,
    lr = 0.1,
    n_layers=1,
    momentum = 0.9,
    dropout=0.5,
)

model_name = str(args)

model = LSTMClassifier(hidden_dims=args.hidden_dims,
                       input_dim=n_features,
                       n_class=n_class,
                       dropout=args.dropout)
loss_fn = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)

/home/minh/.conda/envs/deepset/lib/python3.6/site-packages/torch/nn/modules/rnn.py:46: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [ ]:
experiment_utils.setup_model_logger(_LOGGER, model_name, _MODEL_LOG_ROOT_PATH)

test_losses = experiment_utils.evaluate_validation_loss_template(model, 
                                          loss_fn, 
                                          test_series_df,
                                         test_labels_df)
_LOGGER.info("Initial Validation Loss: {}".format(np.mean(test_losses)))

In [ ]:

training_losses = experiment_utils.train_model_template(model, loss_fn, optimizer,
                                                       args.n_epoch, train_series_df,
                                                       train_labels_df, _LOGGER)
torch.save(model, "{}/{}.pt".format(_MODEL_ROOT_PATH, model_name))

In [ ]:

test_losses = experiment_utils.evaluate_validation_loss_template(model, 
                                          loss_fn, 
                                          test_series_df,
                                         test_labels_df)

_LOGGER.info("Final Validation Loss: {}".format(np.mean(test_losses)))